In [5]:
# importações iniciais padrão FIFA 2025
import pandas as pd
import numpy as np
import os
from datetime import datetime

DATA_DIR = "./data"

In [7]:
payers = pd.read_csv(f"{DATA_DIR}/payers-v1.csv")
terminals = pd.read_csv(f"{DATA_DIR}/seller_terminals-v1.csv")
transactions = pd.read_csv(f"{DATA_DIR}/transactions_train-v1.csv")

In [9]:
payers.head()

,card_hash,card_bin,card_first_transaction
0,6579071715002803,355137,2016-03-20
1,4814157718073,546566,2016-04-09
2,4717474907925,436231,2016-04-17
3,4747611558497388,358161,2016-05-08
4,4689529748023353,655698,2016-06-03


In [10]:
terminals.head()

,terminal_id,latitude,longitude,terminal_operation_start,terminal_soft_descriptor
0,0,-23.601328,-46.707536,2016-05-06,Urban Fiber 132
1,1,-23.505868,-46.668860,2016-06-10,Express Rentals 22
2,2,-23.554972,-46.618841,2016-06-12,Dream Home Decor 4
3,3,-15.851810,-47.946997,2016-06-13,Urban Spa 108
4,4,-2.996113,-59.963312,2016-06-13,Sportswear Hub 93


In [11]:
transactions.head()

,transaction_id,tx_datetime,tx_date,tx_time,tx_amount,card_id,terminal_id,is_transactional_fraud,is_fraud,tx_fraud_report_date
0,1,2018-01-01 00:00:10,2018-01-01,00:00:10,117.12,4983922282101,1113,0,False,NaN
1,2,2018-01-01 00:00:38,2018-01-01,00:00:38,57.70,38435765939628,790,0,False,NaN
2,3,2018-01-01 00:00:38,2018-01-01,00:00:38,41.50,4053973451359964262,157,0,False,NaN
3,4,2018-01-01 00:00:41,2018-01-01,00:00:41,47.72,370458387500772,850,0,False,NaN
4,5,2018-01-01 00:01:46,2018-01-01,00:01:46,14.53,180022431672084,32,0,False,NaN


In [12]:
# convertendo as datas
transactions["tx_datetime"] = pd.to_datetime(transactions["tx_datetime"])
transactions["tx_date"] = pd.to_datetime(transactions["tx_date"])
payers["card_first_transaction"] = pd.to_datetime(payers["card_first_transaction"])
terminals["terminal_operation_start"] = pd.to_datetime(terminals["terminal_operation_start"])

In [13]:
# juntando todos os df´s para ter um principal com todas as features que temos até agora

df = transactions.merge(payers, left_on="card_id", right_on="card_hash", how="left")
df = df.merge(terminals, on="terminal_id", how="left")

print(f"{df.shape}")
df.head()

(4483956, 17)


,transaction_id,tx_datetime,tx_date,tx_time,tx_amount,card_id,terminal_id,is_transactional_fraud,is_fraud,tx_fraud_report_date,card_hash,card_bin,card_first_transaction,latitude,longitude,terminal_operation_start,terminal_soft_descriptor
0,1,2018-01-01 00:00:10,2018-01-01,00:00:10,117.12,4983922282101,1113,0,False,NaN,4983922282101,300262,2017-10-03,-15.705911,-47.873059,2017-11-12,Ifood BBQ Hub 65
1,2,2018-01-01 00:00:38,2018-01-01,00:00:38,57.70,38435765939628,790,0,False,NaN,38435765939628,213171,2017-09-24,-3.027345,-59.991758,2017-08-06,Modern Sofa Store 92
2,3,2018-01-01 00:00:38,2018-01-01,00:00:38,41.50,4053973451359964262,157,0,False,NaN,4053973451359964262,501837,2017-12-02,-23.579366,-46.560141,2017-01-09,Elite Hair Salon 78
3,4,2018-01-01 00:00:41,2018-01-01,00:00:41,47.72,370458387500772,850,0,False,NaN,370458387500772,358083,2017-05-31,-23.548920,-46.613343,2017-08-27,Cyber World 128
4,5,2018-01-01 00:01:46,2018-01-01,00:01:46,14.53,180022431672084,32,0,False,NaN,180022431672084,180088,2017-10-30,-15.792368,-47.884698,2016-08-29,Live Music Hub 41
